In [1]:
# Install necessary libraries for embeddings, visualizations, and FAISS
%pip install sentence_transformers openai unstructured
%pip install plotly langchain-openai langchain_community tiktoken matplotlib
%pip install -Uqqq rich openai tiktoken wandb unstructured tabulate pdf2image chromadb gradio
%pip install linkify-it-py
%pip install pandas
%pip install langchain
%pip install langchain-text-splitters
%pip install sentence_transformers
%pip -qqq install 'crewai[tools]'
%pip -qqq install youtube-transcript-api
%pip -qqq install yt_dlp
%pip install sentence-transformers openai matplotlib plotly


import torch
import shutil
import os
import pandas as pd
import numpy as np
import faiss
import os

from crewai import Agent
from crewai import Crew, Process
from crewai_tools import YoutubeChannelSearchTool
from crewai import Task
from langchain.chat_models.openai import ChatOpenAI
from sentence_transformers import SentenceTransformer
from langchain.docstore.document import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

pythonLocation = shutil.which("python")
pipLocation = shutil.which("pip")

print(f"Python environment at {pythonLocation}")
print(f"Pip environment at {pipLocation}")





Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.125 requires tiktoken<0.8.0,>=0.7.0, but you have tiktoken 0.8.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packag

/Users/vickyyy/anaconda3/envs/capstone/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Python environment at /Users/vickyyy/anaconda3/envs/capstone/bin/python
Pip environment at /Users/vickyyy/anaconda3/envs/capstone/bin/pip


In [2]:

df = pd.read_csv('winemag-data-130k-v2 2.csv')

# Check the structure of the dataset
df.head()
df.shape


# Create 'data' directory if it doesn't exist
if not os.path.exists('data'):
    os.makedirs('data')

# Save the DataFrame as CSV
df.to_csv('data/wine_data_summary.csv', index=False)

# Check the head of the DataFrame to ensure it's saved correctly
df.head()


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [3]:
# Summarize wine information
def summarize_wine_info(row):
    summary = (
        f"Wine Name: {row['title']}\n"
        f"Country: {row['country']}\n"
        f"Region: {row['region_1']}\n"
        f"Variety: {row['variety']}\n"
        f"Price: {row['price']}\n"
        f"Points: {row['points']}\n"
        f"Description: {row['description']}\n"
    )
    return summary

# Apply the summarization function
df['summary'] = df.apply(summarize_wine_info, axis=1)

# Save this as a CSV for easy access later
df.to_csv('data/wine_data_summary.csv', index=False)
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,summary
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,Wine Name: Nicosia 2013 Vulkà Bianco (Etna)\n...
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,Wine Name: Quinta dos Avidagos 2011 Avidagos R...
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,Wine Name: Rainstorm 2013 Pinot Gris (Willamet...
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,Wine Name: St. Julian 2013 Reserve Late Harves...
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,Wine Name: Sweet Cheeks 2012 Vintner's Reserve...


In [4]:

# Wrap each summary into a Document object
documents = [Document(page_content=text) for text in df['summary'].tolist()]

# Define chunk size and overlap
chunk_size = 1000
chunk_overlap = 100

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

# Split the documents into chunks
texts = text_splitter.split_documents(documents)

print(f"You now have {len(texts)} documents in your data.")


# Load the sentence transformer model on GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embedder = SentenceTransformer('all-MiniLM-L6-v2', device=device)

# Choose 100 sample
# Load the sentence transformer model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Randomly select 100 rows without NaN values
sample_df = df.sample(n=100)

# Apply embedding function to the 'text_column' of the sampled DataFrame
sample_df['embedding'] = sample_df['summary'].apply(lambda x: embedder.encode(x))

sample_df.to_csv('sample_wine_embeddings.csv', index=False)

You now have 129971 documents in your data.


## This cell here is going to embedd the whole dataset, however, it will take at least 1.5 hrs to run. If you want to test the demo, you could use the cell below, which randomly selects 2000 rows from the whole dataset and applies embedding function.

In [5]:
# Precompute embeddings for each wine summary
#from sentence_transformers import SentenceTransformer
#from sklearn.metrics.pairwise import cosine_similarity

# Load the sentence transformer model
# embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Embed the wine summaries and store them in the dataframe
#df['embedding'] = df['summary'].apply(lambda x: embedder.encode(x))

In [6]:
def search_wines(query, df, faiss_index, n=10):
    """
    Function to search wines based on a query and return top 'n' results ranked by cosine similarity.

    Parameters:
    - query: The search query string.
    - df: The dataframe containing wine summaries, ratings, reviews, and embeddings.
    - n: The number of top results to return (default is 10).

    Returns:
    - results: List of top 'n' results containing wine name, score, rating, price, and review.
    """
    # Embed the search query
    query_embedding = embedder.encode(query)

    # Calculate cosine similarity between query and each wine's embedding
    df["similarity"] = df["embedding"].apply(lambda x: cosine_similarity([x], [query_embedding]).flatten()[0])

    # Sort the results by similarity and retrieve the top 'n' wines
    sorted_results = df.sort_values("similarity", ascending=False)

    # Prepare a dictionary to store the unique wines by name
    unique_wines = {}

    # Iterate over the sorted results and add unique wines to the result list
    for _, row in sorted_results.iterrows():
        wine_name = row['title']
        if wine_name not in unique_wines:
            unique_wines[wine_name] = {
                "name": row['title'],
                "score": row['similarity'],
                "rating": row['points'],
                "price": row['price'] if pd.notnull(row['price']) else 'Price not available',
                "description": row['description']
            }
        # Stop when you have enough results
        if len(unique_wines) >= n:
            break

    # Return the final list of unique wines
    return list(unique_wines.values())

In [7]:

# Initialize FAISS index
dimension = 384  # The embedding dimension
index = faiss.IndexFlatL2(dimension)

# Add the embeddings to the FAISS index

embeddings = np.array(sample_df['embedding'].tolist())
index.add(embeddings)

# Verify the index has been built
print(f"Number of embeddings added to FAISS index: {index.ntotal}")

index = faiss.read_index('faiss_index.index')

# Define the search query for sample
# sample dataset
query = "recommend some frutty wine"

# Perform the search and print the ranked results
ranked_wines = search_wines(query, sample_df,index,n=3)

# Display results in a readable format
for idx, wine in enumerate(ranked_wines, 1):
    print(f"{idx}. {wine['name']}")
    print(f"   - Score: {wine['score']:.6f}")
    print(f"   - Rating: {wine['rating']}")
    print(f"   - Price: {wine['price']}")
    print(f"   - Description: {wine['description']}\n")


# Step 1: Save the top choice from ranked_wines as the search result
if ranked_wines:
    top_wine = ranked_wines[0]  # Get the first (top) wine
    search_result = f"Wine Name: {top_wine['name']}\nScore: {top_wine['score']:.6f}\nRating: {top_wine['rating']}\nPrice: {top_wine['price']}\nDescription: {top_wine['description']}"
else:
    search_result = "No wine found matching the query."

# Print the search result (for debugging)
print(search_result)

topic_of_interest = search_result
topic_of_interest


Number of embeddings added to FAISS index: 100
1. Foxen 2011 John Sebastiano Vineyard Pinot Noir (Sta. Rita Hills)
   - Score: 0.525149
   - Rating: 89
   - Price: 48.0
   - Description: The most forward and drinkable of Foxen's many new 2011 Pinot Noirs. It's soft and juicy with raspberries and cherry flavors, plus subtleties of cola, sautéed wild mushrooms and sweet, toasty sandalwood. A pretty wine that's perfect with lamb.

2. Stephen Ross 2010 Pinot Noir (Central Coast)
   - Score: 0.515382
   - Rating: 84
   - Price: 24.0
   - Description: Dry and acidic, this has flavors of green mint and sour cherry candy. Not likely to improve with age, but give it an hour in the decanter to let it soften.

3. Armstrong Family 2013 Fronk Red (Columbia Valley (WA))
   - Score: 0.503625
   - Rating: 90
   - Price: 28.0
   - Description: A blend of Cabernet Franc (80%) and Petit Verdot, this is a perfumed wine with high-toned aromas of violets, herbs, baking spices, pencil eraser, barrel notes an

"Wine Name: Foxen 2011 John Sebastiano Vineyard Pinot Noir (Sta. Rita Hills)\nScore: 0.525149\nRating: 89\nPrice: 48.0\nDescription: The most forward and drinkable of Foxen's many new 2011 Pinot Noirs. It's soft and juicy with raspberries and cherry flavors, plus subtleties of cola, sautéed wild mushrooms and sweet, toasty sandalwood. A pretty wine that's perfect with lamb."

In [ ]:


os.environ["OPENAI_API_KEY"] = "INSERT_YOUR_KEY_HERE"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-turbo"
os.environ["OPENAI_API_BASE_URL"] = "https://api.openai.com/v1"


# The tool used by the topic researcher
youtube_tool = YoutubeChannelSearchTool(youtube_channel_handle='@Winefolly')
youtube_tool

# Topic Researcher Agent
topic_researcher = Agent(
    role='Topic Researcher',
    goal='Search for relevant videos on the topic {topic} from the provided YouTube channel',
    verbose=True,
    memory=True,
    backstory="Expert in finding and analyzing relevant content from YouTube channels, specializing in wine introduction topics.",
    tools=[youtube_tool],
    allow_delegation=True,# commond center,
    llm="openai/gpt-4o-mini"
)

# Wine Expert Agent
wine_expert_agent = Agent(
    role='Wine Expert',
    goal='Create a short wine introduction from the transcription provided by the Topic Researcher that capture key points about the general introduction of the grape, how to best drink the wine and the ideal pairings. Use bullet point and highlight important part to keep the contant clear and well-formatted.',
    verbose=True,
    memory=True,
    backstory="Specializes in creating a concise, well-structured, and easy to understand introduction that resonate with the wine-beginner audience.",
    allow_delegation=False,
    llm="openai/gpt-4o-mini"
)

# Social Post Agent
social_post_agent = Agent(
    role='Social Media Post Creator',
    goal='Create a short and engaging social media post from the transcription provided by the Topic Researcher of no more than 70 words.',
    verbose=True,
    memory=True,
    backstory="Expert in crafting concise and engaging social media posts that resonate with a wine-curious audience and include relevant hashtags for maximum visibility.",
    allow_delegation=False,
    llm="openai/gpt-4o-mini"
)

# Tasks
research_task = Task(
    description="Identify and analyze videos on the topic {topic} from the specified YouTube channel.",
    expected_output="A complete word by word report on the most relevant video found on the topic {topic}.",
    agent=topic_researcher,
    tools=[youtube_tool]
)

wine_intro_task = Task(
    description="Create a short introduction of the wine from the transcription provided by the Topic Researcher.",
    expected_output="A markdown-formatted wine introduction",
    agent=wine_expert_agent,
    output_file='wine_intro.md'
)

social_post_task = Task(
    description="Create a social media post summarizing the key points from the transcription provided by the Topic Researcher, including relevant hashtags.",
    expected_output="A markdown-formatted social post",
    agent=social_post_agent,
    output_file='social-post.md'
)

my_crew = Crew(
    agents=[topic_researcher, wine_expert_agent, social_post_agent],
    tasks=[research_task, wine_intro_task, social_post_task],
    verbose=True,
    process=Process.sequential,
    memory=True,
    cache=True,
    max_rpm=500,
    share_crew=True
)

wine_name = topic_of_interest.split("\n")[0].replace("Wine Name: ", "")
formatted_query = f"Give an introduction of {wine_name}?"

result = my_crew.kickoff(inputs={'topic': formatted_query})

Inserting batches in chromadb: 100%|██████████| 1/1 [00:02<00:00,  2.21s/it]
2024-11-24 14:56:18,949 - 140704482666048 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Topic Researcher
## Task: Identify and analyze videos on the topic Give an introduction of Foxen 2011 John Sebastiano Vineyard Pinot Noir (Sta. Rita Hills)? from the specified YouTube channel.


# Agent: Topic Researcher
## Thought: I need to search for relevant videos on the topic of Foxen 2011 John Sebastiano Vineyard Pinot Noir (Sta. Rita Hills) from the specified YouTube channel.
## Using tool: Search a Youtube Channels content
## Tool Input: 
"{\"search_query\": \"Foxen 2011 John Sebastiano Vineyard Pinot Noir Sta. Rita Hills\"}"
## Tool Output: 
Relevant Content:
@Winefolly


# Agent: Topic Researcher
## Thought: Thought: It seems I need to analyze the content further to find the specific video that provides an introduction to Foxen 2011 John Sebastiano Vineyard Pinot Noir (Sta. Rita Hills).
## Using tool: Search a Youtube Channels content
## Tool Input: 
"{\"search_query\": \"Foxen 2011 John Sebastiano Vineyard Pinot Noir\"}"
## Tool Output: 
Relevant Content:
@Winefoll